# Collaborative Filtering - Based Approaches to the RecSys Challenge 2018
## Authors: Peter Kováč, Denis Hoxha
### WU Wien, 6059 Applications of Data Science, SS 2022

This code is an overview and implementation of Collaborative Filtering approaches to the [RecSys Challenge 2018](https://www.recsyschallenge.com/2018/) implemented on (parts of) the [Million Playlist Dataset](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge) provided by Spotify, for which we cite the following source:

- C.W. Chen, P. Lamere, M. Schedl, and H. Zamani. Recsys Challenge 2018: Automatic Music Playlist Continuation. In Proceedings of the 12th ACM Conference on Recommender Systems (RecSys ’18), 2018. 

Over the course of this notebook, we will:

- Pre-process the data provided in a `.json` format
- Implement several track- and playlist-based collaborative filtering algorithms on the dataset for the purpose of automatic playlist continuation
- Generalise the approaches implemented
- Measure the accuracy and precision of the implemented approaches

As already suggested, the RecSys Challenge 2018 focused on automatic playlist continuation - given a playlist, providing recommendations on songs that would fit well with it and "continue" the playlist seamlessly. We provide our take on the challenge here, combining two existing submissions to the challenge:

- Team **Creamy Fireflies**' submission, which finished 4th in the main track of the challenge
    - [GitHub Repository](https://github.com/tmscarla/spotify-recsys-challenge)
    - [Paper](https://re.public.polimi.it/retrieve/handle/11311/1101639/410637/a4-Antenucci.pdf)
        - S. Antenucci, S. Boglio, E. Chioso, E. Dervishaj, S. Kang, T. Scarlatti, and M. F. Dacrema. Artist-driven layering and user’s behaviour impact on recommendations in a playlist continuation scenario. In *Proceedings of the 2018 ACM Recommender Systems Challenge, RecSysChallenge* 2018, Vancouver, BC, Canada, 2018.
- **UC Berkeley Team's** submission, mainly for the purpose of pre-processing of the data
    - [GitHub Repository](https://github.com/vaslnk/Spotify-Song-Recommendation-ML)
    
We will only use the first slice of the dataset, given the computing constraints of our computers, which simply cannot handle the amount of data in the entire dataset. We also did not take the **challenge** dataset into account - instead we tried to emulate the challenges with a given $k$ value in it on the first 1000 playlists of the original dataset.

---

## 1. Pre-Processing

We are given the entire *Million Playlist Dataset* with, as the name suggests, one million Spotify playlists. Given the sheer volume of the data, we will limit ourselves to only using the first 1000 playlists, sticking to the first slice of the dataset, which we load from the `.json` format it is provided in:

In [2]:
import pandas as pd
import numpy as np
import json

dictionary = json.load(open('/Users/kovy/Downloads/spotify_million_playlist_dataset/data/mpd.slice.0-999.json', 'r'))
playlists = pd.DataFrame.from_dict(dictionary['playlists'], orient='columns')
playlists.head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN


As apparent, `tracks` is a column including a list of dictionaries, which are the songs within the playlist, including all their respective metadata:

In [3]:
playlists["tracks"][0][1]

{'pos': 1,
 'artist_name': 'Britney Spears',
 'track_uri': 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak',
 'artist_uri': 'spotify:artist:26dSoYclwsYLMAKD3tpOr4',
 'track_name': 'Toxic',
 'album_uri': 'spotify:album:0z7pVBGOD7HCIB7S8eLkLI',
 'duration_ms': 198800,
 'album_name': 'In The Zone'}

We extract the songs into a new, longer dataframe, that only includes `pid` as a foreign key referencing the "playlists" df:

In [4]:
songPlaylistArray = []
for index, row in playlists.iterrows():
    for track in row['tracks']:
        songPlaylistArray.append([track['track_uri'], track['artist_uri'], track['artist_name'], track['track_name'], 
                                  row['pid']])
songs = pd.DataFrame(songPlaylistArray)
songs.columns = ['trackid', 'artist_uri', 'artist_name', 'track_name', 'pid']
songs

,trackid,artist_uri,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Britney Spears,Toxic,0
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Beyoncé,Crazy In Love,0
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Justin Timberlake,Rock Your Body,0
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Shaggy,It Wasn't Me,0
...,...,...,...,...,...
67498,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,James Arthur,Say You Won't Let Go,999
67499,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,Big Words,The Answer,999
67500,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,Allan Rayman,25.22,999
67501,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Jon Jason,Good Feeling,999


It looks like we've extracted the songs from the `.json` format sufficiently, however, we have to check for duplicates, since there's no reason not to believe a song would appear in more than one playlist:

In [5]:
print("There is a total of", len(list(songs["trackid"].unique())), 
      "unique songs in the dataframe. \nThe most common song appears in the dataframe", 
      list(songs["trackid"].value_counts())[0], "times.")

There is a total of 34443 unique songs in the dataframe. 
The most common song appears in the dataframe 55 times.


:) For anyone interested, the song is: 

In [6]:
print("The most popular song in this slice of the dataframe is", 
      songs[songs.trackid == list(dict(songs["trackid"].value_counts()).keys())[0]].iloc[0]["track_name"], 
     "by", songs[songs.trackid == list(dict(songs["trackid"].value_counts()).keys())[0]].iloc[0]["artist_name"])

The most popular song in this slice of the dataframe is One Dance by Drake


---
## 1.1 The Playlist-Track (Occurrence) Matrix

To conduct collaborative filtering, we must find a metric to calculate the similarity between the songs, using the information contained in the playlists. The first step for this is creating a binary matrix, in which the playlists are the rows and the tracks are the columns, with each value being a binary indicator of whether the song appears in the playlist or not.

This matrix is called either the **Occurrence Matrix** or the **Playlist-Track Matrix** interchangeably. In this code, we will refer to it at `PTM` or `PT_df`.

In [7]:
songs_ptm = songs[["trackid", "artist_name", "track_name", "pid"]]
PTM = songs_ptm.drop(['artist_name', 'track_name'], 
                   axis=1).pivot_table(index=["pid"], 
                                       columns="trackid", aggfunc=lambda x: 1, fill_value=0)

print(PTM.shape)  # 1000 playlists, 34443 unique songs
PTM

(1000, 34443)


trackid,spotify:track:000mA0etY38nKdvf1N04af,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:006AVH7fq061voGXkUiII4,spotify:track:006PJvsr6CyV3JdBf7wiNF,spotify:track:006yrnQMCZpiUgkR612gC8,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00Ci0EXS4fNPnkTbS6wkOh,spotify:track:00CmjeeHvAVKvx3tcIiZTy,spotify:track:00DYRuYJQzfI6dH4Adkimo,spotify:track:00DlEKhhlQNtjnJk7xqB9O,...,spotify:track:7znZvX0Mt6NBmaI8VCPurT,spotify:track:7zo8XAMYBG6nGpqGiIudBc,spotify:track:7zprfY9KDG8g3S5BIOB7xZ,spotify:track:7zqLBFKCBkk5IfbgKgH4VZ,spotify:track:7zscdQe9CjzXnqT3P1Ey7K,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zuwaenG5AF0vG7o7kMduX,spotify:track:7zxRMhXxJMQCeDDg0rKAVo,spotify:track:7zz1drChhd4hQBiGSnLRBZ,spotify:track:7zzBEZBTJejWeL6EqWmCD9
pid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.1.1 BM25 Transformation

In order to use this matrix as a similarity measure, we have to apply the $BM25$ [transformation](https://en.wikipedia.org/wiki/Okapi_BM25) a to a matrix akin to a [Document-Term Matrix](https://en.wikipedia.org/wiki/Document-term_matrix) in NLP.

For this, we first have to turn the dataframe into a *Compressed Sparse Row* Matrix (`csr`), on which we then perform the following operation for each song/playlist (Similar to TF-IDF, or IDF-TF in this order):

$$
\sum_{i=1}^n ln \left( \frac{N - n(s_i) + 0.5}{n(s_i) + 0.5} +1 \right) * \frac{f(s_i,P) \cdot (k_1+1)}{f(s_i,P) + k_1 \cdot \left(1 - b + b \cdot \frac{| P |}{avg|P|} \right)}
$$

Where the symbols represent:

- $N$: Number of playlists
- $n(s_i)$: Number of playlists song $s_i$ appears in
- $f(s_i, P)$: The frequency of song $i$ in a  givenplaylist (in this case $\in \{ 0, 1 \}$)
- $k_1,b$: Arbitrary coefficients we choose by hand, usually $k \sim 1.25$, $b \sim 0.75$
- $|P|$ and $avg|P|$ are the given and average playlist length respectively

This code is inspired by [this](https://github.com/arosh/BM25Transformer/blob/master/bm25.py) github library, after whose functions we modelled the following transformation of the `PTM`:

In [8]:
import scipy
import scipy.sparse as sp
from scipy.sparse import csr_matrix

A = csr_matrix(PTM)
X=A

# Starting points
use_idf=True
k1=1.25 # Coefficient k 
b=0.75 # Coefficient b
n_samples, n_features = X.shape # Number of PLAYLISTS & SONGS

# IDF metric - Prerequisites
if use_idf==True:
    df = np.sum(X, axis=0) # The SONG FREQUENCY VECTOR
    idf = np.log((n_samples - df + 0.5) / (df + 0.5)) # IDF of all the Songs!
    _idf_diag = sp.spdiags(idf, diags=0, m=n_features, n=n_features) # Diagonal Sparse Matrix

# BM25 Operation
dl = X.sum(axis=1) # Number of (UNIQUE) songs in each playlist, (1000x1 Matrix)
sz = X.indptr[1:] - X.indptr[0:-1] # Number of non-zero elements for each playlist (1000, Array)
rep = np.repeat(np.asarray(dl), sz) # Repeat "dl" for "sz" times in each row
avgdl = np.average(dl) # Average document length (amount of songs per playlist - 66.72)

# BM25 Application
data = X.data * (k1 + 1) / (X.data + k1 * (1 - b + b * rep / avgdl)) # BM25 score (non-zero elements)
X = sp.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

# IDF metric - Application
if use_idf==True:
    X = X * _idf_diag # Multiplying each SONG COLUMN by its IDF score (diagonal matrix)
    
X

<1000x34443 sparse matrix of type '<class 'numpy.float64'>'
	with 66721 stored elements in Compressed Sparse Row format>

In [9]:
PT_df= pd.DataFrame(data = X.toarray(), index = list(PTM.index), columns = PTM.columns)
PT_df

trackid,spotify:track:000mA0etY38nKdvf1N04af,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:006AVH7fq061voGXkUiII4,spotify:track:006PJvsr6CyV3JdBf7wiNF,spotify:track:006yrnQMCZpiUgkR612gC8,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00Ci0EXS4fNPnkTbS6wkOh,spotify:track:00CmjeeHvAVKvx3tcIiZTy,spotify:track:00DYRuYJQzfI6dH4Adkimo,spotify:track:00DlEKhhlQNtjnJk7xqB9O,...,spotify:track:7znZvX0Mt6NBmaI8VCPurT,spotify:track:7zo8XAMYBG6nGpqGiIudBc,spotify:track:7zprfY9KDG8g3S5BIOB7xZ,spotify:track:7zqLBFKCBkk5IfbgKgH4VZ,spotify:track:7zscdQe9CjzXnqT3P1Ey7K,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zuwaenG5AF0vG7o7kMduX,spotify:track:7zxRMhXxJMQCeDDg0rKAVo,spotify:track:7zz1drChhd4hQBiGSnLRBZ,spotify:track:7zzBEZBTJejWeL6EqWmCD9
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Looking at `PT_df` (after the transformation) in comparison to `PTM` (before the transformation), we see that the 0's stay 0's - but the 1's have changed accordingly. The $IDF$ part of the BM25 transformation punishes common songs by giving them lower scores and accentuates less common songs that still appear by assigning them higher scores. The $TF$ part of the transformation changes the values only ever so slightly, according to the coefficients and adjusting for playlist length.

In [10]:
print("The song", list(songs[songs.trackid == "spotify:track:006AVH7fq061voGXkUiII4"].track_name)[0], 
      "by", list(songs[songs.trackid == "spotify:track:006AVH7fq061voGXkUiII4"].artist_name)[0], 
      "appears in playlist with pid =", list(songs[songs.trackid == "spotify:track:006AVH7fq061voGXkUiII4"].pid)[0], "once.",
     "\nIt's score after BM25 is, however:", round(list(PT_df["spotify:track:006AVH7fq061voGXkUiII4"])[-1],3),
     "- Much higher than 1. \nThis is also because the song only appears in", sum(PTM["spotify:track:006AVH7fq061voGXkUiII4"]),"playlist in total.")

The song Fingertips by Ryan O'Shaughnessy appears in playlist with pid = 999 once. 
It's score after BM25 is, however: 7.577 - Much higher than 1. 
This is also because the song only appears in 1 playlist in total.


## 1.2 Co-Occurrence Matrix

One last step we need in pre-processing (for later) is the creation of a co-occurrence matrix, which is a symmetrical indicator of the number of playlists songs $i$ and $j$ appear in together. For this, we need $A$, the original `PTM` in the `csr`, *Compressed Sparse Row* format. The Co-Occurrence matrix $C$ is calculated by simply multiplying $A * A^T $:

In [11]:
C = A.T * A # The co-occurrence matrix is simply the occurrence matrix transposed times itself
print(C.shape) # Song x Song
C.toarray() # Matrix of song co-occurrence

(34443, 34443)


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 4, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

---
# 2. Item/Track-Based Collaborative Filtering

## 2.1 Jaccard Index

We start by replicating part of the [UC Berkeley Submission](https://github.com/vaslnk/Spotify-Song-Recommendation-ML/blob/master/EDA.ipynb), which uses the **Jaccard Index** as a measure of similarity to generate songs recommendations for a given playlist. This is the first and simplest instance of collaborative filtering we'll use in this notebook. The [Jaccard Index](https://en.wikipedia.org/wiki/Jaccard_index), in our case, is a form of track-to-track approach. Simply put, we want to calculate the similarity between the songs by using $\frac{| \{p_i\} \cap \{p_j\}|}{| \{p_i\} \cup \{p_j\}|}$, where $\{p_n\}$ is the set of playlists that include the song $n$. 

What we basically want to compute with this index is the ratio of the **intersection** of the playlists including the two songs we're comparing (a.k.a. the playlists in which *both* the songs are located) and the **union** of the playlists including the two songs (a.k.a. the playlists in which *either* of the songs is located).

For this purpose, we will employ our `PTM` matrix along with the *co-occurrence* ($C$) matrix we have construced above. Our co-occurrence matrix $C$ will serve as the numerator of the similarity formula:

$$S_{ij} = \frac{| \{p_i\} \cap \{p_j\}|}{| \{p_i\} \cup \{p_j\}|}$$

We then need to calculate the denominator. Knowing the theory behind it we know that $| A \cup B| = |A| + |B| - |(A \cap B)|$. In order to calculate this, we use the `df` metric from the $BM25$ transformation, which we've already calculated and which indicates the number of playlists a song appears in:

In [12]:
for i in range(4,6):
    print("The", str(i+1)+"th", "song appears in", 
          df.tolist()[0][i], "playlists.") # And so on...
    
df_matrix = df.reshape(34443, 1)
df_matrix # We reshape it for matrix algebra

The 5th song appears in 1 playlists.
The 6th song appears in 6 playlists.


matrix([[1],
        [2],
        [1],
        ...,
        [4],
        [1],
        [1]], dtype=int64)

Applying the theory, we can re-write our similarity matrix as: 

$$S_{ij} = \frac{| \{p_i\} \cap \{p_j\}|}{|\{p_i\}| + |\{p_j\}| - | \{p_i\} \cap \{p_j\}|}$$

...and compute it as follows:

In [13]:
C1 = C.toarray()
D  = (df_matrix + df_matrix.T)-C1
S1 = np.divide(C1, D)

In [14]:
S1

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]])

Ta-da, we now have the **Similarity Matrix**, which calculates the Jaccard similarity between all the songs in this slice of the dataset.

Going forward, we will splice this matrix to identify the indices of the songs in a given playlist (which will take the place of ROWS), while the COLUMNS (all songs) will stay the same. We will then calculate the sum of their similarities as a way of finding fitting songs to continue a playlist.

We generalise this process in a simple function `jaccard_recommender`. According to the competition rules, the function should take the playlist metadata (in our case simply the `pid` argument) and a `k` number of seed tracks from the playlist, under the condition $k \in \{0,1,5,10,25,100\}$. The function will then return 500 songs, given a playlist ID and k randomly sampled tracks from a playlist.

*Note: If $k > |p|$, the next highest $k$ is taken. If $k=0$, the most popular songs are taken.*

In [15]:
sim = pd.DataFrame(data = S1, index = PTM.columns, columns = PTM.columns) # Similarity Matrix as a DataFrame
jaccard_songs = songs_ptm # Songs to be recommended from

# Functions (and packages) we need:
import bisect
from bisect import bisect_right

def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect_right(a, x)
    if i:
        return a[i-1]
    raise ValueError
    
import operator

def jaccard_recommender(playlist_id, k):
    k_option = {0, 1, 5, 10, 25, 100}
    
    # Setting a condition for the values of k (outer-most loop):
    if k in k_option:
        
        # A list of (non-duplicate) songs in playlist that we choose:
        playlist = jaccard_songs[jaccard_songs.pid==playlist_id].drop_duplicates(subset="trackid") 
        
        # Random sample of k songs from the chosen playlist:
        # First, defining k (in case there's less than k songs in the playlist):
        test_list = [0, 1, 5, 10, 25, 100]
        if k > len(playlist):
            k = find_le(test_list, len(playlist))
        else:
            k=k
        
        # Sampling the songs:
        playlist_sample = playlist.sample(n=k,axis='rows')
        
        # Slicing the similarity matrix to have only our songs in the rows:
        sim_slice = sim.loc[list(playlist_sample.trackid)]
        
        # Creating the recommendations:
        similarity_score = np.sum(sim_slice.to_numpy(), axis=0) #  Column-wise sum of all the songs (Summing all rows by column)
        songs_scores = dict(zip(list(sim.columns), similarity_score.tolist())) # DICTIONARY of the songs with id's
        songs_scores_sorted = dict(sorted(songs_scores.items(), key=operator.itemgetter(1), reverse=True)) # Sorting
        
        # We take the first 500
        top_500 = list(songs_scores_sorted.keys())[:500+len(playlist_sample)] # Take the first 500 + the legnth of the OG playlist sample
        recommended_songs = jaccard_songs.loc[(jaccard_songs["trackid"].isin(top_500))] # Filter the songs
        recommended_songs = recommended_songs[["trackid", "artist_name", "track_name", "pid"]].drop_duplicates(subset="trackid") # Deduplicate
        
        # Remove the songs WITHIN the sample:
        result = recommended_songs[recommended_songs.trackid.isin([c for c in list(recommended_songs.trackid) if c not in list(playlist_sample.trackid)])]
        
        return result

    else:
        raise ValueError("jaccard_recommender: status must be one of %r." % k_option)

And we can see the function working on playlist 0 and $k=25$:

In [16]:
jaccard_recommender(0, 25)

,trackid,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0
7,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Buttons,0
8,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,Say My Name,0
10,spotify:track:2gam98EZKrF9XuOkU13ApN,Nelly Furtado,Promiscuous,0
...,...,...,...,...
54331,spotify:track:3kS9f8YsnypXn5xGu90uHa,The Cheetah Girls,Cheetah Sisters,812
54332,spotify:track:1g1Jor1zrllXn2ogj8KGAH,The Cheetah Girls,"Strut - From ""The Cheetah Girls 2""",812
54337,spotify:track:5Plb8xps8VZRF2sXlrPi3Q,Raven-Symoné,That's So Raven (Theme Song),812
57643,spotify:track:5y4TDQdYzlT4eoQIPOgNDz,Pretty Ricky,Your Body,854


As is apparent, the function suggests songs that *are* within the given playlist (in this case 0), but *are not* within the 25 song sample that the function takes using $k$.

If $k=0$:

In [17]:
# k = 0
print("If k=0, the recommendations for any playlist are", 
      (sum(jaccard_recommender(0, 0).trackid == jaccard_recommender(500, 0).trackid)/500)*100, 
      "% the same - the most popular songs are simply given.")

If k=0, the recommendations for any playlist are 100.0 % the same - the most popular songs are simply given.


## 2.2 kNN Track-Based Collaborative Filtering

The other option for track-based collaborative filtering is using $kNN$ and `PTM` to calculate the similarity scores between two songs, as done by [Creamy Fireflies](https://re.public.polimi.it/retrieve/handle/11311/1101639/410637/a4-Antenucci.pdf). In this case, the score prediction of track $i$ for playlist $u$ is calculated as:

$$r_{ui} = \sum_{j \in u}^{KNN} r_{uj} * (s_{ji})^p$$

Where $p$ is an arbitrary normalisation coefficient and KNN are the top k-nearest neighbors of said song/playlist. In this case, since we're only dealing with one slice, we can set this to the total number of songs in our dataset slice.

$s_{ij}$, the similarity between tracks is calculated as the dot-product of two columns of `PTM`:

$$ s_{ij} = r_i * r_j$$

We will basically sum up the playlist rating of each song in the given playlist by the similarity with the suggested song to the power of $p$. After that, we will pick the 500 songs with the highest score.

In order not to have to calculate the dot product every time, we circumvent it using linear algebra to calculate it for each pair of songs, akin to what we did in the co-occurrence matrix above, giving us a 34443x34443 dataframe containing the similarity between each song pair in the dataset:

In [18]:
ptm_csr = csr_matrix(PT_df) # Create a sparse matrix
simes = ptm_csr.T*ptm_csr # Multiply it by it's transposed versino
simes_df = pd.DataFrame(data = simes.toarray(), index = list(PTM.columns), columns = PTM.columns) # Create DF
simes_df

trackid,spotify:track:000mA0etY38nKdvf1N04af,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:006AVH7fq061voGXkUiII4,spotify:track:006PJvsr6CyV3JdBf7wiNF,spotify:track:006yrnQMCZpiUgkR612gC8,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00Ci0EXS4fNPnkTbS6wkOh,spotify:track:00CmjeeHvAVKvx3tcIiZTy,spotify:track:00DYRuYJQzfI6dH4Adkimo,spotify:track:00DlEKhhlQNtjnJk7xqB9O,...,spotify:track:7znZvX0Mt6NBmaI8VCPurT,spotify:track:7zo8XAMYBG6nGpqGiIudBc,spotify:track:7zprfY9KDG8g3S5BIOB7xZ,spotify:track:7zqLBFKCBkk5IfbgKgH4VZ,spotify:track:7zscdQe9CjzXnqT3P1Ey7K,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zuwaenG5AF0vG7o7kMduX,spotify:track:7zxRMhXxJMQCeDDg0rKAVo,spotify:track:7zz1drChhd4hQBiGSnLRBZ,spotify:track:7zzBEZBTJejWeL6EqWmCD9
spotify:track:000mA0etY38nKdvf1N04af,92.231565,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
spotify:track:000xQL6tZNLJzIrtIgxqSl,0.000000,64.389018,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
spotify:track:006AVH7fq061voGXkUiII4,0.000000,0.000000,57.409078,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
spotify:track:006PJvsr6CyV3JdBf7wiNF,0.000000,0.000000,0.000000,9.809952,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
spotify:track:006yrnQMCZpiUgkR612gC8,0.000000,0.000000,0.000000,0.000000,49.214224,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:7zsw78LtXUD7JfEwH64HK2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,59.825945,0.000000,0.000000,0.00000,0.000000
spotify:track:7zuwaenG5AF0vG7o7kMduX,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,24.484046,0.000000,0.00000,0.000000
spotify:track:7zxRMhXxJMQCeDDg0rKAVo,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,62.704992,0.00000,0.000000
spotify:track:7zz1drChhd4hQBiGSnLRBZ,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,22.11166,0.000000


In [19]:
#songs_ptm # Songs to be recommended from

def knn_recommender(playlist_id, k):
    k_option = {0, 1, 5, 10, 25, 100}
    
    # Setting a condition for the values of k (outer-most loop):
    if k in k_option:
        
        # A list of (non-duplicate) songs in playlist that we choose:
        playlist = songs_ptm[songs_ptm.pid==playlist_id].drop_duplicates(subset="trackid") 
        
        # Random sample of k songs from the chosen playlist:
        # First, defining k (in case there's less than k songs in the playlist):
        test_list = [0, 1, 5, 10, 25, 100]
        if k > len(playlist):
            k = find_le(test_list, len(playlist))
        else:
            k=k
        
        # Sampling the songs:
        playlist_sample = playlist.sample(n=k,axis='rows')
        
        # Rating of the song within the playlist
        r_uj = PT_df[list(playlist_sample.trackid)].iloc[playlist_id] # First part of the score
        
        # Slicing the columns from the playlist sample (k)
        sim_slice = simes_df[list(playlist_sample.trackid)]
        
        # Multiplying each row of the df by the same value - r_uj
            # This might seem reduntant, but it makes our code scalable
        r_uj_df = pd.DataFrame(data=[r_uj])
        r_uj_df_0 = pd.concat([r_uj_df]*simes_df.shape[0]).sort_index()
        r_uj_df_0 = r_uj_df_0.set_index(simes_df.columns)
        
        sim_slice_edited = r_uj_df_0*sim_slice
        
        # Creating the recommendations:
        similarity_score = np.sum(sim_slice_edited.to_numpy(), 
            axis=1) # SIMILARITY: Row-wise sum of all the songs (Summing all columns by row)
        songs_scores = dict(zip(list(simes_df.columns), similarity_score.tolist())) # DICTIONARY of the songs with id's
        songs_scores_sorted = dict(sorted(songs_scores.items(), key=operator.itemgetter(1), reverse=True)) # Sorting
        
        # We take the first 500
        top_500 = list(songs_scores_sorted.keys())[:500+len(playlist_sample)] # Take the first 500 + the legnth of the OG playlist sample
        recommended_songs = songs_ptm.loc[(songs_ptm["trackid"].isin(top_500))] # Filter the songs
        recommended_songs = recommended_songs[["trackid", "artist_name", "track_name", "pid"]].drop_duplicates(subset="trackid") # Deduplicate
        
        # Remove the songs WITHIN the sample:
        result = recommended_songs[recommended_songs.trackid.isin([c for c in list(recommended_songs.trackid) if c not in list(playlist_sample.trackid)])]
        
        return result

    else:
        raise ValueError("knn_recommender: status must be one of %r." % k_option)

In [20]:
knn_recommender(0,25)

,trackid,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0
5,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Yeah!,0
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0
...,...,...,...,...
52203,spotify:track:0J1Wfjo9H3R62yaCuuNDZX,Hilary Duff,So Yesterday,779
54254,spotify:track:1uHt85NYsb2JJ05FkAK6as,High School Musical Cast,Get'Cha Head In The Game,812
61089,spotify:track:1nIDYHvLaZnD5lEf66SH80,Bella Thorne,TTYLXOX,908
61115,spotify:track:0jEwNhGaid3bhzpXk6pMHr,Usher,Burn - Radio Mix,908


---

# 3. Playlist-Based Collaborative Filtering

## 3.1 Tversky Coefficient

In this form of collaborative filtering, we will employ the [Tversky Index](https://en.wikipedia.org/wiki/Tversky_index) to calculate the similarity between playlists. This measure is similar to the *Jaccard Index*, serving as its generalisation and is defined by [Creamy Fireflies](https://re.public.polimi.it/retrieve/handle/11311/1101639/410637/a4-Antenucci.pdf) in their paper as:

$$
s_{ij} = \frac{r_i * r_j}{\alpha(|r_i| - r_i * r_j) + \beta(|r_j| - r_i * r_j) + r_i * r_j + h}
$$

In this case, $r_i$ and $r_j$ represent the **rows** of `PTM`, telling us that this similarity measure compares playlists rather than songs themselves.

We demonstrate how such a coefficient is calculated between two playlists and apply it to a few of our given playlists to give us a taste of a symmetrical tversky similarity matrix of 1000x1000 playlists:

In [21]:
def tversky(p1, p2, a=1, b=1, h=0.2): # Tanimoto coefficient/Jaccard Index
    
    # Enumerator
    enumerator = np.dot(PT_df.iloc[p1],PT_df.iloc[p2]) # Enumerator
    # Denominator
    denominator = a*(sum(PT_df.iloc[p1] != 0) - enumerator) + b*(sum(PT_df.iloc[p2] != 0) - enumerator) + enumerator + h

    tversky = enumerator/denominator
    return tversky

rows = []
for j in range(0,3):
    jako=[]
    for i in range(0,PT_df.shape[0]):
        ii = tversky(j, i)
        jako.append(ii)
    rows.append(jako)

pd.DataFrame(rows, columns = range(0,PT_df.shape[0]))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-1.073274,0.000000,0.000000,0.0,0.0,0.52236,0.0,0.0,0.0,0.0,...,0.712714,0.0,1.281981,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,-1.045837,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,1.274482,0.0,0.0,0.0,0.0
2,0.000000,0.000000,-1.049281,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


Now that we have defined the *Tversky* similarity for our playlists, the recommending process becomes much simpler. We use the following formula to predict a rating of song $i$ for a given playlist $u$:

$$
r_{ui} = \sum_{i \in v}^{KNN} (s_{uv})^p * r_{vi}
$$

Where:
- $i \in v$ indicates *all* the playlists the song $i$ occurs in
- $s_{uv}$ is the similarity of the given playlist and the playlist $i$ occurs in
- $p$ is the arbitrary power coefficient
- $r_{vi}$ is the rating of song $i$ in the playlist $v$

We compute these metrics for all of the songs and pick out the 500 most relevant ones for our given playlist.

In [22]:
# Generalisation - A rating for all songs for a given playlist
def tversky_rating(pid):
    # Playlist Component
    tversky_pid=[]
    for i in range(0,PT_df.shape[0]):
        ii = tversky(pid, i)
        tversky_pid.append(ii)
        
    # Song component
    rating = []
    for j in range(0, PT_df.shape[1]):
        r_vi = PT_df[PT_df.columns[j]]
        rating.append(sum(tversky_pid*r_vi))
        
    return rating

In [23]:
def tversky_recommender(playlist_id, k):
    k_option = {0, 1, 5, 10, 25, 100}
    
    # Setting a condition for the values of k (outer-most loop):
    if k in k_option:
        
        # A list of (non-duplicate) songs in playlist that we choose:
        playlist = songs_ptm[songs_ptm.pid==playlist_id].drop_duplicates(subset="trackid") 
        
        # Random sample of k songs from the chosen playlist:
        # First, defining k (in case there's less than k songs in the playlist):
        test_list = [0, 1, 5, 10, 25, 100]
        if k > len(playlist):
            k = find_le(test_list, len(playlist))
        else:
            k=k
        
        # Sampling the songs:
        playlist_sample = playlist.sample(n=k,axis='rows')
        
        songs_scores = dict(zip(list(PT_df.columns), # DICTIONARY of the songs with id's
                                tversky_rating(playlist_id))) # Based on the tversky coefficient
        songs_scores_sorted = dict(sorted(songs_scores.items(), key=operator.itemgetter(1), reverse=True)) # Sorting
        
        # We take the first 500
        top_500 = list(songs_scores_sorted.keys())[:500+len(playlist_sample)] # Take the first 500 + the legnth of the OG playlist sample
        recommended_songs = songs_ptm.loc[(songs_ptm["trackid"].isin(top_500))] # Filter the songs
        recommended_songs = recommended_songs[["trackid", "artist_name", "track_name", "pid"]].drop_duplicates(subset="trackid") # Deduplicate
        
        # Remove the songs WITHIN the sample:
        result = recommended_songs[recommended_songs.trackid.isin([c for c in list(recommended_songs.trackid) if c not in list(playlist_sample.trackid)])]
        result = result.iloc[0:500] # Remove the additional songs
        return result

    else:
        raise ValueError("tversky_recommender: status must be one of %r." % k_option)

In [24]:
tversky_recommender(0,25)

,trackid,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0
32,spotify:track:4E5P1XyAFtrjpiIxkydly4,Iyaz,Replay,0
42,spotify:track:5lDriBxJd22IhOH9zTcFrV,The All-American Rejects,Dirty Little Secret,0
...,...,...,...,...
52176,spotify:track:4Ctmbw1hJks6cPrr6x8FbC,No Doubt,Underneath It All,779
52179,spotify:track:0it4iEbCYlWUfSM2LEy1gh,Shakira,Underneath Your Clothes,779
52184,spotify:track:0q3lwFiwQYQynThTulLKGN,Blu Cantrell,Breathe - Rap Version,779
52185,spotify:track:0cv1Mnb47oMvjVKMc2cyZw,Christina Aguilera,Can't Hold Us Down,779


---

# 4. Results Evaluation 

### Metrics:
- **R-Precision**: Similar to recall, binary accuracy measure (recommended/relevant)

$$R = \frac{| G \cap R_{1:|G|}|}{|G|}$$


- **NDCG**: Order of suggested songs taken into account

$$NDCG = \frac{rel_1 + \sum_{i=2}^{|R|} \frac{rel_i}{log_2 i}}{1 + \sum_{i=2}^{|G \cap R |} \frac{1}{log_2 i}}$$

## 4.1 R-Precision

We start with the **R-Precision** metric, which is defined as "the number of retrieved relevant tracks divided by the number of known relevant tracks". Since we don't have access to the "holdout tracks" defined by Spotify, we define the terms as follows:

- **Retrieved relevant tracks**: The number of tracks retrieved from the *rest* of the playlist not present in the $k$ tracks sampled to make the prediction
- **Known relevant tracks**: The total number of tracks in the sampled playlist minus $k$

*Caution: The statistic will vary each time, since the $k$ songs are randomly sampled each time. We will therefore average it accross the entire dataset for each of the recommender methods*

In [129]:
k=1 # We choose this value of k to accomodate smaller playlists too

r_precision_jaccard = []

for i in range(0, PTM.shape[0]):
    submission_jaccard = jaccard_recommender(i, k)
    n = sum(submission_jaccard.pid==i)
    d = songs[songs.pid==i].drop_duplicates(subset="trackid").shape[0] - k
    
    r_precision_jaccard.append(n/d)
    
r_precision_knn = []

for i in range(0, PTM.shape[0]):
    submission_knn = knn_recommender(i, k)
    n = sum(submission_knn.pid==i)
    d = songs[songs.pid==i].drop_duplicates(subset="trackid").shape[0] - k
    
    r_precision_knn.append(n/d)

In [132]:
print("R precision:\n - Jaccard Recommender:", round(sum(r_precision_jaccard)/len(r_precision_jaccard),4)*100, "%",
     "\n - kNN Recommender:", round(sum(r_precision_knn)/len(r_precision_knn), 4)*100, "%")

R precision:
 - Jaccard Recommender: 49.51 % 
 - kNN Recommender: 51.05 %


We see that both the track-based recommenders, that is the Jaccard and kNN recommender systems do similarly well, returning an R-precision of about 50%. This is, of course, not comparable to the results achieved by the teams in RecSys Challenge 2018 - our methods are, after all, different. Both recommenders however perform very well, especially with such a low value of $k=1$. This can also be attributed to the small size of the sample we perform our analysis on, where many songs repeat themselves and song similarity therefore plays a very heavy role in creating recommendations.

We did not compute this metric for the playlist-based Tversky recommender due to scalability issues - Such a measurement is simply not possible with our machines in the given time for that function:

In [134]:
import time
tmp = time.time()
submission_tversky = tversky_recommender(100, 1)
tmp1 = time.time() - tmp

print("Measuring 1000 playlists with the tversky recommender would take roughly", 
      round(((tmp1*1000)/60)/60, 2), "hours.")

Measuring 1000 playlists with the tversky recommender would take roughly 5.24 hours.


## 4.2 NDCG

The *Normalised discounted cumulative gain measure is defined as:*

$$ DCG = rel_1 + \sum_{i=2}^{|R|} \frac{rel_i}{log_2 i} $$

$$ IDCG = 1 + \sum_{i=2}^{|G \cap R |} \frac{1}{log_2 i} $$

$$ NDCG = \frac{DCG}{IDCG}$$


$DCG$, the Discounted Cumulative Gain measures the ranking quality of the recommended tracks and is divided by the $IDCG$, the Ideal Discounted Cumulative gain as a normalising measure.

We use the $NDCG$ function as developed by team [hojinYang](https://github.com/hojinYang/spotify_recSys_challenge_2018), whose submission ended up 2nd in the RecSys Challenge 2018:

In [135]:
import math 

def get_ndcg(answer, cand):
    cand_len = len(cand) 
    idcg=1
    idcg_idx=2
    dcg=0
    if cand[0] in answer:  dcg=1
    
    for i in range(1,cand_len):
        if cand[i] in answer: 
            dcg += (1/math.log(i+1,2))
            idcg += (1/math.log(idcg_idx,2))
            idcg_idx+=1
    
    return dcg/idcg

And we repeat the same procedure we did for R-Precision with NDCG:

In [154]:
k=1

jaccard_ndcg = []

for i in range(0, PTM.shape[0]):
    jaccard_prediction = jaccard_recommender(i,k)
    a = list(songs[songs.pid==i].drop_duplicates(subset="trackid").trackid)
    c = list(jaccard_prediction.trackid)
    jaccard_ndcg.append(get_ndcg(a,c))
    
knn_ndcg = []

for i in range(0, PTM.shape[0]):
    knn_prediction = knn_recommender(i,k)
    a = list(songs[songs.pid==i].drop_duplicates(subset="trackid").trackid)
    c = list(knn_prediction.trackid)
    knn_ndcg.append(get_ndcg(a,c))

In [159]:
print("NDCG:\n - Jaccard Recommender:", round(sum(jaccard_ndcg)/len(jaccard_ndcg),4)*100, "%",
     "\n - kNN Recommender:", round(sum(knn_ndcg)/len(knn_ndcg), 4)*100, "%")

NDCG:
 - Jaccard Recommender: 51.42 % 
 - kNN Recommender: 52.32 %


We see that, once again, the *Normalised Discounted Cumulative Gain* reaches high levels in both cases, being slightly higher for $kNN$. Everything said about R-precision, along with it not being comparable to the challenge results and being too straining for the Tversky recommender applies in this case as well.